RFC, Recursive Feature Elimination is mostly used for large dataset, implements wrapper method that uses model to select the features, so a model must have a way to calculate the features, for example: Decision Trees. You can also use SelectKBest(f_classif,k=100) or regularization for linear models, or can use PCA for dimensionality reduction

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [32]:
df = pd.read_csv('F:\dataset_for_ML\supervised\chatGptGenerated\classification_dataset_big.csv')

In [33]:
df.shape

(5000, 1001)

In [3]:
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,target
0,-1.198470,-0.457443,-13.633427,-0.655549,-0.792618,1.963241,-1.654373,-2.316143,0.818318,-1.347614,...,0.283207,-0.928814,-26.885736,-0.632304,0.101181,-0.843080,-1.276564,1.099012,0.751698,0
1,1.742162,1.148670,-9.433289,-0.687951,0.829379,-0.379140,0.430317,-2.581936,2.489849,0.781945,...,-0.178653,-0.844384,3.305692,0.803635,-1.080531,-0.596931,0.862293,-1.266792,0.672638,0
2,0.041822,0.047625,-3.084783,2.478229,2.041862,-0.286272,-0.739128,-0.926777,-0.327327,0.004213,...,-2.792606,-0.159659,-19.192704,-0.206836,-0.682471,0.465222,1.445831,-1.171759,2.061527,0
3,0.949004,1.775962,8.190364,-0.463841,-1.842682,0.094567,-1.061304,0.372592,-0.477794,-1.369563,...,0.131179,-0.639487,-13.057277,0.810640,1.699888,-0.380029,-1.583151,0.409173,0.608763,1
4,-1.936809,-0.548025,-14.298078,-0.883577,0.217931,0.383131,0.421256,0.917669,2.195287,-0.033925,...,-0.410378,-1.436348,-29.510306,1.312613,0.741027,-0.632904,0.657839,1.155401,0.438823,0


In [4]:
df.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,target
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,0.013816,-0.012874,-2.420857,-0.015094,-0.004441,0.015754,0.019624,-0.018185,0.010033,0.009673,...,-0.001125,0.025648,-2.133091,0.002864,-0.003110,0.006371,0.000231,0.005092,0.024146,0.49960
std,0.997734,0.997454,14.806836,0.996346,0.991291,0.993386,1.025185,0.998698,1.010179,1.003832,...,0.997530,1.005510,17.989210,1.000415,0.991986,0.989826,0.999543,0.998262,0.994879,0.50005
min,-3.584143,-3.723584,-54.441804,-3.907993,-3.763148,-3.431294,-3.911546,-3.435108,-3.554716,-3.813362,...,-3.546712,-3.827281,-65.718681,-3.855024,-3.710596,-4.230121,-3.598875,-3.693187,-3.912116,0.00000
25%,-0.647841,-0.687469,-12.368298,-0.705328,-0.680018,-0.673983,-0.674700,-0.688540,-0.672855,-0.630928,...,-0.673756,-0.657513,-14.291468,-0.674695,-0.677777,-0.659765,-0.679122,-0.679700,-0.656385,0.00000
50%,0.005424,-0.010823,-2.548493,-0.020172,0.000987,0.018425,0.018100,-0.010520,0.030639,0.004331,...,-0.002745,0.030754,-1.909087,0.004401,0.008389,0.005137,0.014488,-0.005569,0.027264,0.00000
75%,0.683372,0.661698,7.393885,0.674766,0.655667,0.695895,0.719174,0.647127,0.690564,0.692744,...,0.668827,0.709304,9.932319,0.674464,0.644917,0.665525,0.680188,0.672849,0.727323,1.00000
max,3.769819,3.296677,54.000114,3.406120,3.351738,4.239758,4.484078,3.428898,3.367043,3.616735,...,3.387285,3.563579,78.357840,3.522124,3.359297,3.606929,3.746400,3.408192,3.603367,1.00000


Here, just performing rfe, no z-score calculation, plotting,nothing. Once i know exactly how to ,i will definitely get back to make this more efficient, right now, i do not have so much expertise/experience in this field! I know a bit about taking a random sample of the dataset, but let's not do that right now ! Also i am not using hyperparameter tuning for DT. I am just doing it in raw data and the dataset is generated BY AI

In [11]:
lr_model = LogisticRegression()
dtc = DecisionTreeClassifier(random_state=42)

In [12]:
X = df.iloc[:,:-1]
y = df["target"]

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [ ]:
# def feature_selection_func(X_train,X_test,y_train,y_test,model):
#     accuracy_score_list,recall_score_list,precision_score_list,f1_score_list = [],[],[],[]
#     for i,column in enumerate(X_train.columns,1):
#         # print(i,column)
#         selector = RFE(estimator=model,n_features_to_select=i,step=100)
#         selector.fit(X_train,y_train)
#         X_train_selector = selector.transform(X_train)
#         X_test_selector = selector.transform(X_test)

#         model.fit(X_train_selector,y_train)
#         pred = model.predict(X_test_selector)

#         f1score = round(f1_score(y_test,pred,average="weighted"),3)        
#         accuracyscore = round(accuracy_score(y_test,pred),3)        
#         recallscore = round(recall_score(y_test,pred,average="weighted"),3)        
#         precisionscore = round(precision_score(y_test,pred,average="weighted"),3)  

#         f1_score_list.append(f1score)      
#         accuracy_score_list.append(accuracyscore)      
#         recall_score_list.append(recallscore)      
#         precision_score_list.append(precisionscore) 

#     return accuracy_score_list,f1_score_list,recall_score_list,precision_score_list

In [34]:
from sklearn.feature_selection import RFECV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import StratifiedKFold

def feature_selection_func(X_train, X_test, y_train, y_test, model):
    rfecv = RFECV(
        estimator=model,
        step=50,  # removes 50 features per iteration
        cv=StratifiedKFold(5),
        scoring='accuracy',  # or 'f1_weighted' if preferred
        n_jobs=-1
    )
    rfecv.fit(X_train, y_train)

    # Transform data using selected features
    X_train_selected = rfecv.transform(X_train)
    X_test_selected = rfecv.transform(X_test)

    # Fit the model again on selected features
    model.fit(X_train_selected, y_train)
    pred = model.predict(X_test_selected)

    # Metrics
    f1score = round(f1_score(y_test, pred, average="weighted"), 3)
    accuracyscore = round(accuracy_score(y_test, pred), 3)
    recallscore = round(recall_score(y_test, pred, average="weighted"), 3)
    precisionscore = round(precision_score(y_test, pred, average="weighted"), 3)

    return accuracyscore, f1score, recallscore, precisionscore, rfecv.support_


First, let's check the model without feature selection

In [35]:
lr_model.fit(X_train,y_train)

c:\Users\DELL\.conda\envs\pandasNumpyMatplotlibPrac\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [36]:
pred = lr_model.predict(X_test)

In [37]:
accuracy_score(pred,y_test),precision_score(pred,y_test),recall_score(pred,y_test),f1_score(pred,y_test)

(0.739, 0.7211740041928721, 0.7288135593220338, 0.7249736564805058)

In [38]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier(random_state=42)

In [39]:
pred1 = dtc.predict(X_test)

In [40]:
accuracy_score(pred1,y_test),recall_score(pred1,y_test),precision_score(pred1,y_test),f1_score(pred1,y_test)

(0.688, 0.6680244399185336, 0.6876310272536688, 0.6776859504132231)

In [42]:
accuracyscores,f1scores,recallscores,precisionscores = feature_selection_func(X_train,X_test,y_train,y_test,dtc)

ValueError: too many values to unpack (expected 4)